In [1]:
########29/09/2022测试版本
#######对于每一个在库中的sku根据网页链接进行爬取内容
######此方法爬取了bohm网站中的产品sku的子sku信息
#####input : all_sku, all_url 产品母sku以及对应母链接
#####output :  子sku的详细信息

#####03/10/2022测试成功，解决了产品只有一个主图的情况









#####库
from bs4 import BeautifulSoup  # this module helps in web scrapping.
import urllib.request
import re
import pandas as pd
from pandas.core.frame import DataFrame
from openpyxl.writer.excel import ExcelWriter
from lxml import etree
import requests  # 模拟浏览器请求的库
from ast import Break
import json

header = {
    'user-agent': 'Mozilla/5.0(Macinatosh;Intel Mac OS X 10_13_6)Applewebkit/537.36(KHTML,like Gecko)Chrome/81.0.4044.138 Safari/537.36',
}

In [2]:
#之后会考虑将它移出去

###allurl sku对应供应商的url
###all——name sku对应的url命名

##这些东西都可以在附图情况表中找到
##input 是sku对应的url链接 sku名字  以及sku本身
##output是一个含所有variant的excel，我们将它导入到airtable里面后面进行第二部操作

all_url=[

"https://bohm-paris.com/products/bracelet-bohm-paris-betty",
"https://bohm-paris.com/products/bracelet-bohm-paris-bea",
"https://bohm-paris.com/products/bracelet-bohm-paris-hanna",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-pauline",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-pauline-argente",
"https://bohm-paris.com/products/bracelet-bohm-paris-spiros",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-adreline",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-lea",
"https://bohm-paris.com/products/bracelet-bohm-paris-amonet",
"https://bohm-paris.com/products/bracelet-bohm-paris-djen",
"https://bohm-paris.com/products/bracelet-bohm-paris-djeser",
"https://bohm-paris.com/products/bracelet-bohm-paris-penellie",
"https://bohm-paris.com/products/bracelet-bohm-paris-penellie-argent",
"https://bohm-paris.com/products/bracelet-bohm-paris-peneloppe",
"https://bohm-paris.com/products/bracelet-bohm-paris-siloe",
"https://bohm-paris.com/products/bracelet-bohm-paris-peneloppe-argente",
"https://bohm-paris.com/products/bracelet-bohm-paris-sohan",
"https://bohm-paris.com/products/bracelet-bohm-paris-esther",
"https://bohm-paris.com/products/bracelet-bohm-paris-haby",
"https://bohm-paris.com/products/bracelet-bohm-paris-lois",
"https://bohm-paris.com/products/bracelet-bohm-paris-zackary",
"https://bohm-paris.com/products/bracelet-bohm-paris-adventure",
"https://bohm-paris.com/products/bracelet-bohm-paris-thiago",
"https://bohm-paris.com/products/bracelet-bohm-paris-thymeo",
"https://bohm-paris.com/products/bracelet-bohm-paris-rebecca",
"https://bohm-paris.com/products/bracelet-bohm-paris-star",
"https://bohm-paris.com/products/bracelet-bohm-paris-priscie",
"https://bohm-paris.com/products/bracelet-bohm-paris-naim",
"https://bohm-paris.com/products/bracelet-bohm-paris-noemie",
"https://bohm-paris.com/products/bracelet-bohm-paris-loric",
"https://bohm-paris.com/products/bracelet-bohm-paris-maylon",
"https://bohm-paris.com/products/bracelet-bohm-paris-etolila",
"https://bohm-paris.com/products/bracelet-bohm-paris-talisman-happiness",
"https://bohm-paris.com/products/bracelet-bohm-paris-daphinia",
"https://bohm-paris.com/products/bracelet-bohm-paris-dreamer",
"https://bohm-paris.com/products/bracelet-bohm-paris-talisman-carpe-diem",
"https://bohm-paris.com/products/bracelet-bohm-paris-chleia",
"https://bohm-paris.com/products/bracelet-bohm-paris-anastasy",
"https://bohm-paris.com/products/bracelet-bohm-paris-armand",
"https://bohm-paris.com/products/bracelet-bohm-paris-adisson",
"https://bohm-paris.com/products/bracelet-bohm-paris-talisman-family-love",
"https://bohm-paris.com/products/bracelet-wild-rock",
"https://bohm-paris.com/products/bracelet-bohm-paris-talisman-love",
"https://bohm-paris.com/products/bracelet-bohm-paris-amael",
"https://bohm-paris.com/products/bracelet-bohm-paris-azurion",
"https://bohm-paris.com/products/bracelet-bohm-paris-cybelle",
"https://bohm-paris.com/products/bracelet-cheville-bohm-paris-loni",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-lion",
"https://bohm-paris.com/products/bracelet-bohm-paris-khalissa",
"https://bohm-paris.com/products/bracelet-bohm-paris-lonni",
"https://bohm-paris.com/products/bracelet-bohm-paris-lupin",
"https://bohm-paris.com/products/bracelet-bohm-paris-milandra",
"https://bohm-paris.com/products/bracelet-bohm-paris-ziggy",
"https://bohm-paris.com/products/bracelet-bohm-paris-locker-victoria",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-wael",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-lennon",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-sadio",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-kelian",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-lansana",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-joevin",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-evan",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-geban",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-jaden",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-etally",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-hevan",
"https://bohm-paris.com/products/bracelet-jonc-bohm-paris-ayden",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-corantin",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-florentio",
"https://bohm-paris.com/products/bracelet-bohm-paris-moana",
"https://bohm-paris.com/products/bracelet-bohm-paris-sixtine",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-loulita",
"https://bohm-paris.com/products/bracelet-bohm-paris-nikki",
"https://bohm-paris.com/products/bracelet-bohm-paris-malina",
"https://bohm-paris.com/products/bracelet-bohm-paris-braceline",
"https://bohm-paris.com/products/bracelet-bohm-paris-shera",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-manoel-dore",
"https://bohm-paris.com/products/bracelet-bohm-paris-lukas",
"https://bohm-paris.com/products/bracelet-bohm-paris-lassan",
"https://bohm-paris.com/products/bracelet-bohm-paris-aubin",
"https://bohm-paris.com/products/bracelet-bohm-paris-adelie",
"https://bohm-paris.com/products/bracelet-bohm-paris-loman-cube",
"https://bohm-paris.com/products/bracelet-bohm-paris-hans",
"https://bohm-paris.com/products/bracelet-bohm-paris-loman-triangle",
"https://bohm-paris.com/products/bracelet-bohm-paris-lyllian",
"https://bohm-paris.com/products/bracelet-bohm-paris-lassana",
"https://bohm-paris.com/products/bracelet-bohm-paris-feelia",
"https://bohm-paris.com/products/bracelet-bohm-paris-darina",
"https://bohm-paris.com/products/bracelet-bohm-paris-wiktoria",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-jerome",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-sonny",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-mason",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-masonie",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-ronie",
"https://bohm-paris.com/products/bracelet-bohm-paris-jonc-nomi",
"https://bohm-paris.com/products/bracelet-bohm-paris-ambroisine",
"https://bohm-paris.com/products/bracelet-bohm-paris-gonzague",
"https://bohm-paris.com/products/bracelet-bohm-paris-vinciane-perles",
"https://bohm-paris.com/products/bracelet-bohm-paris-dispy",
"https://bohm-paris.com/products/bracelet-bohm-paris-devon",
"https://bohm-paris.com/products/bracelet-bohm-paris-bobby",
"https://bohm-paris.com/products/bracelet-bohm-paris-galen",
"https://bohm-paris.com/products/bracelet-bohm-paris-carlota",
"https://bohm-paris.com/products/bracelet-bohm-paris-cameo",
"https://bohm-paris.com/products/bracelet-bohm-paris-dianthe",
"https://bohm-paris.com/products/bracelet-bohm-paris-pricia",
"https://bohm-paris.com/products/bracelet-bohm-paris-calandra",
"https://bohm-paris.com/products/bracelet-bohm-paris-mailine",
"https://bohm-paris.com/products/bracelet-bohm-paris-yesenia",
"https://bohm-paris.com/products/bracelet-bohm-paris-azelina",
"https://bohm-paris.com/products/bracelet-bohm-paris-dylane",
"https://bohm-paris.com/products/bracelet-bohm-paris-ariadnanie",
"https://bohm-paris.com/products/bracelet-bohm-paris-christabel",
"https://bohm-paris.com/products/bracelet-jonc-bohm-paris-abriana",
"https://bohm-paris.com/products/bracelet-cheville-bohm-paris-ashantie",
"https://bohm-paris.com/products/bracelet-bohm-paris-ariadne-1",
"https://bohm-paris.com/products/bracelet-cheville-bohm-paris-azelie",
"https://bohm-paris.com/products/bracelet-cheville-bohm-paris-aelis",
"https://bohm-paris.com/products/bracelet-bohm-paris-lurilla",
"https://bohm-paris.com/products/bracelet-jonc-bohm-paris-chloefina",
"https://bohm-paris.com/products/bracelet-jonc-bohm-paris-phoebe",
"https://bohm-paris.com/products/bracelet-jonc-bohm-paris-phoebe-argente",
"https://bohm-paris.com/products/bracelet-bohm-paris-maryse",
"https://bohm-paris.com/products/bracelet-jonc-bohm-paris-abatina",




]


all_sku=[
    
"BMBR00269N",
"BMBR00271N",
"BMBR00276N",
"BMBR00283N",
"BMBR00291N",
"BMBR00313N",
"BMBR00329N",
"BMBR00331N",
"BMBR00360N",
"BMBR00362N",
"BMBR00363N",
"BMBR00368N",
"BMBR00369N",
"BMBR00370N",
"BMBR00371N",
"BMBR00378N",
"BMBR00379N",
"BMBR00401N",
"BMBR00465N",
"BMBR00466N",
"BMBR00467N",
"BMBR00468N",
"BMBR00469N",
"BMBR00470N",
"BMBR00471N",
"BMBR00472N",
"BMBR00473N",
"BMBR00474N",
"BMBR00475N",
"BMBR00476N",
"BMBR00477N",
"BMBR00478N",
"BMBR00479N",
"BMBR00480N",
"BMBR00481N",
"BMBR00482N",
"BMBR00483N",
"BMBR00484N",
"BMBR00485N",
"BMBR00486N",
"BMBR00487N",
"BMBR00488N",
"BMBR00518N",
"BMBR00531N",
"BMBR00532N",
"BMBR00533N",
"BMBR00534N",
"BMBR00535N",
"BMBR00536N",
"BMBR00538N",
"BMBR00539N",
"BMBR00541N",
"BMBR00543N",
"BMBR00568N",
"BMBR00571N",
"BMBR00572N",
"BMBR00573N",
"BMBR00574N",
"BMBR00575N",
"BMBR00576N",
"BMBR00577N",
"BMBR00578N",
"BMBR00579N",
"BMBR00580N",
"BMBR00581N",
"BMBR00582N",
"BMBR00583N",
"BMBR00584N",
"BMBR00627N",
"BMBR00638N",
"BMBR00669N",
"BMBR00682N",
"BMBR00683N",
"BMBR00684N",
"BMBR00688N",
"BMBR00702N",
"BMBR00710N",
"BMBR00719N",
"BMBR00728N",
"BMBR00770N",
"BMBR00772N",
"BMBR00773N",
"BMBR00774N",
"BMBR00775N",
"BMBR00776N",
"BMBR00792N",
"BMBR00793N",
"BMBR00794N",
"BMBR00830N",
"BMBR00831N",
"BMBR00832N",
"BMBR00833N",
"BMBR00834N",
"BMBR00835N",
"BMBR00837N",
"BMBR00839N",
"BMBR00868N",
"BMBR00891N",
"BMBR00945N",
"BMBR00946N",
"BMBR00950N",
"BMBR00951N",
"BMBR01030N",
"BMBR01031N",
"BMBR01034N",
"BMBR01038N",
"BMBR01040N",
"BMBR01043N",
"BMBR01044N",
"BMBR01045N",
"BMBR01051N",
"BMBR01052N",
"BMBR01055N",
"BMBR01062N",
"BMBR01064N",
"BMBR01066N",
"BMBR01068N",
"BMBR01074N",
"BMBR01142N",
"BMBR01160N",
"BMBR01162N",
"BMBR01163N",
"BMBR01177N",


]

all_name=[


"bracelet-jonc-acier-inoxydable",
"bracelet-fin-acier-inoxydable",
"jonc-bracelet",
"bracelet-jonc-femme-tendance",
"bracelet-en-jonc",
"bracelet-en-chaine",
"bracelet-jonc-fantaisie",
"bracelet-jonc-acier-femme",
"bracelet-de-pierre-naturelle",
"bracelet-perles-fines",
"bracelet-chaine-doré",
"bracelet-maille-trombone",
"bracelet-maille-carrée",
"bracelet-chaine-acier-inoxydable",
"Bracelet-Chaîne-en-Acier-Inoxydable",
"Bracelet-Chaîne-en-Maille",
"bracelet-fin-doré-femme",
"bracelet-chaine-acier-inoxydable",
"bracelet-acier-inoxydable-doré-femme",
"bracelet-fantaisie-doré",
"bracelet-chaine-fine-femme",
"bracelet-maille-rectangle",
"bracelet-gros-maillon-femme",
"bracelet-chaine-maillon",
"bracelet-maillon-femme",
"bracelet-etoile-filante",
"bracelet-chaine-femme-acier-inoxydable",
"Bracelet-Chaine-Entrelacée",
"Bracelet-Chaîne-Maillons-de-Barre",
"Bracelet-Noeud-Chaîne",
"Bracelet-Chaîne-Anneaux-Entrelacés",
"Bracelet-Fin-Etoile",
"Bracelet-Fin-Barre",
"Bracelet-Fin-Multirang",
"Bracelet-Fin-Double-Rang",
"Bracelet-Fin-Cylindre",
"Bracelet-Fin-Maillon",
"Bracelet-Chaine-Maillon-Anneau-Ovale",
"Bracelet-Chaine-Trombone-et-Noeud",
"Bracelet-Anneaux-Chaîne",
"Bracelet-Double-Chaine-avec-Barre",
"Bracelet-Double-Rang-Fin-avec-Barre",
"Bracelet-Fin-avec-Barre",
"petit-bracelet-fin-femme",
"bracelet-fin-pour-femme",
"bracelet-femme-pierre-naturelle",
"bracelet-de-cheville-femme",
"bracelet-jonc-acier-inoxydable-femme",
"petit-bracelet-fin",
"Bracelet-Fin-à-Frange",
"Bracelet-Fin-Frange-et-Chaînette",
"bracelets-fins-femme",
"bracelet-fin-acier-femme",
"Bracelet-Maillon-en-Coeur",
"Bracelet-Jonc-Couronne",
"Jonc-Couronne",
"Bracelet-Jonc-Ajouré",
"Bracelet-Jonc-Couronne-Ouvert",
"Jonc-Couronne-Ouvert",
"Bracelet-Jonc-Ajouré-Ouvert",
"Jonc-Manchette-Ajouré-ouvert",
"Jonc-Manchette-Couronne-Ajouré-ouvert",
"Jonc-Manchette-Feuille",
"Jonc-Manchette-Etoile",
"Jonc-Manchette-Motif-Fleur",
"Jonc-Manchette-Multi-Rangs",
"Jonc-Manchette-Lotus",
"Bracelet-Jonc-Feuille",
"bracelet-en-perles-heishi",
"Bracelet-Maillon-Anneaux-Entrelacés",
"Jonc-Doré-Femme",
"bracelet-rondelle-en-pierre-naturelle",
"bracelet-avec-médaillon-ovale-royal",
"bracelet-femme-fin-fantaisie",
"bracelet-en-rondelle-acier-inoxydable",
"bracelet-jonc-torsadé-ouvert",
"bracelet-coquillages-et-pierres-naturelles",
"bracelet-chaine-maillon-entrelacé",
"Bracelet-en-Anneaux-Entrelacés",
"Bracelet-Maillon-en-acier-inoxydable",
"Bracelet-avec-Pampilles-Acier-Inoxydable",
"Bracelet-Maillons-avec-Pendentif-Coeur",
"Bracelet-avec-Pampilles-Triangles",
"Bracelet-Maillon-avec-Perle-d'eau",
"bracelet-chaine-avec-coeur",
"Bracelet-Maillon-Entrelacé-en-Acier-Inoxydable",
"Bracelet-Trombone-Aneau-Entrelacé",
"Bracelet-3-Chaînes-en-Acier-Inxydable",
"bracelet-jonc-ajouré-avec-boules-nacrées",
"bracelet-jonc-ajouré-double-rangs",
"bracelet-jonc-Couronne-Feuille-ajourée",
"jonc-Couronne-Feuille-ajourée",
"jonc-Couronne-Royal-ajourée",
"jonc-simple-avec-pampilles",
"Bracelet-Maillons-et-Perles-d'eau-Douce",
"Bracelet-Demi-Mallions-Demi-Perles",
"Bracelet-Mailles-et-Perles-d'eau-douce",
"Bracelet-en-Perles-Rondelles-Acier",
"Bracelet-Fin-avec-Pendentif-Carré",
"Bracelet-Fin-avec-Pendentif-Rond",
"Bracelet-Maille-Fantaisie-2-Rangs",
"Bracelet-Fil-Pierre-Naturelle-et-Acier",
"Bracelet-Chaine-avec-Médaillon-Réversible",
"Bracelet-Fin-avec-Pierre-Naturelle-Ovale",
"Bracelet-en-Perles-et-Pierres-Petites",
"Bracelet-Fin-Double-Rangs-Médaillon-Rond",
"Bracelet-Elastique-en-Perles-Rondelles",
"Bracelet-Fin-avec-Petit-Coeur-en-Email",
"Bracelet-Elastique-en-Perles-Teintées",
"Bracelet-Elastique-en-Perles-Emaillées",
"Bracelet-Elastique-en-Perles-Rondes",
"Bracelet-Elastique-en-Perles-Crématiques",
"Bracelet-Jonc-4-Rangs-avec-Pierres",
"Bracelet-de-Cheville-Perles-de-Coquillages",
"Bracelet-de-Chevilles-sur-Fil-Perles-Rondes",
"Bracelet-Cheville-sur-Fil-Perles-Rondelles",
"Bracelet-de-Cheville-en-perles-Tintées",
"Bracelet-Elastique-Perles-Céramique-Acier-Inox",
"Bracelet-Fin-Double-Rang-avec-Pierre",
"Jonc-Manchette-Simple-avec-Cristaux",
"Bracelet-Jonc-Simple-Deux-Cristaux",
"Bracelet-Fin-avec-Pampilles-Rondes",
"Jonc-Multirangs-avec-Pierre-Naturelle",


]


print(len(all_url))

123


In [3]:
###对每个网址进行爬取

# 休眠时判定
from ast import main
import json
from time import sleep
ROOTSKUS=[]
ROOTURLS=[]
BOHMSKUS=[]
VALNAMES=[]
VALLINKS=[]
VALIMGS=[]



grabcount = 1

####读取数据流
for i in range(0, len(all_url)):
    print("开始做第%d个sku"%(i+1))
    grabcount += 1
 
    # 100的整数倍
    if (grabcount%100 == 0):
        print("休息一下")
        sleep(120)
    try:
        soup = BeautifulSoup(requests.get(all_url[i]).text, "html.parser")
        #json数据格式
        jsontest = soup.find("script",{"type":"application/json"})
        jsonstr=jsontest.string
        jsondata=json.loads(jsonstr)
        #一方面可以获得所有子sku的数量，另一方面可以判断网站数据是否可以被探索到
        rootskulen=len(jsondata.get("variants"))
        for rootindex in range(0,rootskulen):
            ##bohmskus      print(str(jsondata.get("offers")[rootindex].get("sku")))
            bohmsku=str(jsondata.get("variants")[rootindex].get("sku"))
            ##valnames       print(str(jsondata.get("offers")[rootindex].get("name")))
            valname=str(jsondata.get("variants")[rootindex].get("title"))
            ##urllinks      bohm网站类型中vallink的形式是母sku名+子id
            vallink=str(all_url[i])+"?variant="+str(jsondata.get("variants")[rootindex].get("id"))
            #print(vallink)
            ###可能找不到图片，我们要测试
            try:
                valimg=jsondata.get("variants")[rootindex].get("featured_image").get("src")
                #写入
                print("成功找到图片，准备写入")
                ROOTSKUS.append(all_sku[i])
                ROOTURLS.append(all_url[i])
                BOHMSKUS.append(bohmsku)
                VALNAMES.append(valname)
                VALLINKS.append(vallink)
                VALIMGS.append(valimg)
            except:
                print("使用第一种方法未成功找到图片现在尝试第二种方法")
                #一般对应图片只有一种的情况，option内的值会设置成null
                try:
                    valimg=jsondata.get("featured_image")
                    print("第二种方法成功找到，准备写入")
                    valimg='https:'+valimg
                    print(valimg)
                    ###没有请求头，加个头再添加
                    
                    ROOTSKUS.append(all_sku[i])
                    ROOTURLS.append(all_url[i])
                    BOHMSKUS.append(bohmsku)
                    VALNAMES.append(valname)
                    VALLINKS.append(vallink)
                    VALIMGS.append(valimg)
                except:
                    print("真的找不到图片了")
                    ROOTSKUS.append(all_sku[i])
                    ROOTURLS.append(all_url[i])
                    BOHMSKUS.append(bohmsku)
                    VALNAMES.append(valname)
                    VALLINKS.append(vallink)
                    VALIMGS.append("notfound")

                


    except:
        print("无法获取产品信息")
        ROOTSKUS.append(all_sku[i])
        ROOTURLS.append(all_url[i])
        BOHMSKUS.append("notfound")
        VALNAMES.append("notfound")
        VALLINKS.append("notfound")
        VALIMGS.append("notfound")
    






开始做第1个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第2个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第3个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第4个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第5个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第6个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第7个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第8个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写入
开始做第9个sku
成功找到图片，准备写入
成功找到图片，准备写入
成功找到图片，准备写

In [4]:
res=DataFrame({"根sku":ROOTSKUS,"根链接":ROOTURLS,"bohm的sku":BOHMSKUS,"sku属性":VALNAMES,"sku属性对应链接":VALLINKS,"sku属性对应图片":VALIMGS})

In [5]:
res

,根sku,根链接,bohm的sku,sku属性,sku属性对应链接,sku属性对应图片
0,BMBR00269N,https://bohm-paris.com/products/bracelet-bohm-...,BR-02A-70,vert foncé,https://bohm-paris.com/products/bracelet-bohm-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
1,BMBR00269N,https://bohm-paris.com/products/bracelet-bohm-...,BR-02A-71,vert paon,https://bohm-paris.com/products/bracelet-bohm-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
2,BMBR00269N,https://bohm-paris.com/products/bracelet-bohm-...,BR-02A-72,Ocean,https://bohm-paris.com/products/bracelet-bohm-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
3,BMBR00269N,https://bohm-paris.com/products/bracelet-bohm-...,BR-02A-73,rose fuchsia,https://bohm-paris.com/products/bracelet-bohm-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
4,BMBR00269N,https://bohm-paris.com/products/bracelet-bohm-...,BR-02A-74,rouge,https://bohm-paris.com/products/bracelet-bohm-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
...,...,...,...,...,...,...
494,BMBR01177N,https://bohm-paris.com/products/bracelet-jonc-...,BMB15470680,Onyx,https://bohm-paris.com/products/bracelet-jonc-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
495,BMBR01177N,https://bohm-paris.com/products/bracelet-jonc-...,BMB15470680,agate blanche,https://bohm-paris.com/products/bracelet-jonc-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
496,BMBR01177N,https://bohm-paris.com/products/bracelet-jonc-...,BMB15470680,Tourmaline,https://bohm-paris.com/products/bracelet-jonc-...,https://cdn.shopify.com/s/files/1/0076/0961/44...
497,BMBR01177N,https://bohm-paris.com/products/bracelet-jonc-...,BMB15470680,Turquoise africaine,https://bohm-paris.com/products/bracelet-jonc-...,https://cdn.shopify.com/s/files/1/0076/0961/44...


In [6]:
res.to_excel('11102022sku_detail.xlsx',index=False)

In [ ]:
####得出来的东西放airtable上面来进行url信息的查询   然后命名也可以随之查出
####接下来，input 为查出来的结果  
####output 为直接下载图片到文件夹中

In [10]:
####尝试整理excel中的图片


from asyncio.windows_events import NULL
from bs4 import BeautifulSoup  # this module helps in web scrapping.
import requests  # this module helps us to download a web page 
from pandas.core.frame import DataFrame
from email.mime import image
from openpyxl import load_workbook
import pandas as pd
import re
import urllib.request
###先读取数据######将airtable中的数据放到excel中.####sku 图片名字 图片链接
df = pd.read_excel('data_mainimage.xlsx', sheet_name='Sheet1')

##图片下载地址
savePath = 'D:\mainimage12102022//'

In [11]:
sumline=len(df)
sumline

2

In [12]:
for i in range(0, sumline):
    ###每一行
    nowrow=df.values[i]
    imgname=nowrow[1]+'.jpg'
    imgurl=nowrow[2]
    try:
        
        urllib.request.urlretrieve(imgurl,savePath+imgname)
    except:
        print("%s链接无效，可能是为空或者换链接了"%(imgname))
        print(imgurl)
        pass



In [ ]:
#########下面的均为logs



In [ ]:
savePath = 'D:\mainimage11102022//'
imgname="Bracelet-en-Perles-et-Pierres-Petites-MalachiteHowlite-bleue.jpg"
imgurl="https://cdn.shopify.com/s/files/1/0076/0961/4403/products/bracelet-bohm-paris-dianthe-bijoux-bohm-paris-onyx-825782.jpg?v=1652558823"
urllib.request.urlretrieve(imgurl,'D:\picture//'+imgname)

In [ ]:


#type(jsontest)
#bs4.element.Tag
#rootlinkname['href']   获取标签属性可以直接用索引查找
#rootlinkname.string    可以直接将标签内部内容化为字符串
#rootlinkname=soup.find('link',{"rel": "canonical"})   产品的根链接
#由于图片链接很阴间，所以要进行二次爬取
print("检索子sku完成")
print("一共要爬取%d个图片\n \n"%(len(VALLINKS)))
print("开始爬取图片链接\n")

grabcount2=1
for i2 in range(0,len(VALLINKS)):
    print("开始找第%d个图片"%(i2+1))
    grabcount2 += 1
 
    # 100的整数倍
    if (grabcount2%100 == 0):
        print("休息一下")
        sleep(120)
    try:
        nowurl=VALLINKS[i2]
        soup=BeautifulSoup(requests.get(nowurl).text, "html.parser")
        ####取有参数的第一项（网页中的第一张图片）
        valimg=str(soup.find('img',{"class":"Image--lazyLoad Image--fadeIn","data-original-src": True})['data-original-src'])
        VALIMGS.append(valimg)
        

    except:
        print("爬取图片时出现问题，可能是url无效")
        VALIMGS.append("NOTFOUND")



In [ ]:
######soup
soup=BeautifulSoup(requests.get(VALLINKS[1],headers=header).text, "html.parser")

In [ ]:

#找到了第一张图的算法
imgval=soup.find('img',{"class":"Image--lazyLoad Image--fadeIn","data-original-src": True})

In [ ]:
 #nowurl=VALLINKS[i2]
soup=BeautifulSoup(requests.get("https://bohm-paris.com/products/coffret-bohm-paris-boucles-hadley").text, "html.parser")
        ####取有参数的第一项（网页中的第一张图片）
valimg=soup.find_all("div",{"class":"AspectRatio AspectRatio--withFallback"})
#valimg=soup.find_all("noscript")
#valimg[0]["data-srcset"]

In [ ]:
valimg=soup.find("script",{"type":"application/json"}).string



In [ ]:
valimgdata=json.loads(valimg)
valimgdata.get("variants")


In [ ]:
len(valimgdata.get("variants"))

In [ ]:
valimgdata

In [ ]:
valimgdata.get("variants")[1].get("featured_image").get("src")

In [ ]:
valimg="//cdn.shopify.com/s/files/1/0076/0961/4403/products/bague-lounis-bague-bohm-paris-642780.jpg?v=1631108396"
valimg='https:'+valimg
print(valimg)
###ok